In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd drive/MyDrive/CS6220/Data/

/content/drive/.shortcut-targets-by-id/1WVaZ7FJ-T7E_GxiFFZDUa5-6me-HB0uf/CS6220/Data


In [3]:
!ls

ensemble_results  results_ensemble.json  results.json  Transformed_2p  val_2p


In [4]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.models import vgg16, resnet152, alexnet
import torch

import os
import json
from PIL import Image
import numpy as np

import random
np.random.seed(0)
random.seed(0)
torch.manual_seed(0)

In [5]:
args = {"batch_size": 32}
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda:0


In [6]:
class ImageNetDataset(Dataset):
    def __init__(self, image_dir, annotations_file, transformations = None, device = 'cpu'):
        self.image_dir = image_dir
        self.images = os.listdir(self.image_dir)
        self.annotations = json.load(open(annotations_file))
        self.transformations = transformations
        self.device = device

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.images[idx])
        img = Image.open(img_path).convert('RGB')
        label = torch.tensor(int(self.annotations[self.images[idx].split('.')[0]]))
        if self.transformations:
            img = self.transformations(img)
        #img = img.to(self.device)
        return self.images[idx].split('.')[0].split('_')[-1], img, label


preprocessing = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]),
])

In [7]:
def compute_accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))
    res = []
    for k in topk:
        correct_k = correct[:k].contiguous().view(-1).float().sum(0)
        res.append(correct_k)
    return res

In [8]:
from tqdm import tqdm
def automate(modelName, directory):
  imagenetValDataset = ImageNetDataset(image_dir = directory, 
                                    annotations_file='../Data/val_2p/imagnet_classes.json', 
                                    transformations=preprocessing, 
                                    device = device)
  imagenetValDataloader = torch.utils.data.DataLoader(imagenetValDataset, batch_size=args['batch_size'], shuffle=True, num_workers=4)

  if(modelName == "alexnet"):
    model = alexnet(pretrained = True)
  elif(modelName == "vgg16"):
    model = vgg16(pretrained = True)
  else:
    model = resnet152(pretrained = True)
  
  model.eval()
  model.to(device)
    
  accuracy_top1 = 0
  accuracy_top5 = 0
  count = 0
  allPreds = {}
  for batch_idx, (imgname, img, label) in enumerate(tqdm(imagenetValDataloader)):
      img = img.to(device)
      label = label.to(device)
      out = model(img).detach()
      outc = out.cpu().numpy()
      bs = out.shape[0]
      for i in range(bs):
        allPreds[imgname[i]] = outc[i]
      count += out.shape[0]
      top1, top5 = compute_accuracy(out, label, topk = (1, 5))
      accuracy_top1 += top1
      accuracy_top5 += top5
      if(batch_idx % 100 == 0):
        
        print(modelName, " Evaluated {}/{}: Top 1 Accuracy: {:.3f} Top 5 Accuracy: {:.3f}".format(str(count), 
                                                         str(len(imagenetValDataset)), accuracy_top1*100.0/count, accuracy_top5*100.0/count))
        
  print(modelName, " Evaluated {}/{}: Top 1 Accuracy: {:.3f} Top 5 Accuracy: {:.3f}".format(str(count), 
                                                         str(len(imagenetValDataset)), accuracy_top1*100.0/count, accuracy_top5*100.0/count))
  return allPreds, accuracy_top1*100.0/count, accuracy_top5*100.0/count

In [9]:
import os
import csv
import pandas as pd
directoryName = os.listdir("Transformed_2p/transform/")
modelNames = ["alexnet", "vgg16", "resnet152"]

for i in modelNames:
  for j in directoryName:
    if(not os.path.exists('./ensemble_results/{}_{}.csv'.format(i, j))):
      allPreds, top1, top5 = automate(i, "../Data/Transformed_2p/transform/" + j)
      df = pd.DataFrame.from_dict(allPreds).transpose()
      df.to_csv('./ensemble_results/{}_{}.csv'.format(i, j))
      print("Done for {}_{}".format(i, j))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

  3%|▎         | 1/32 [00:08<04:18,  8.35s/it]

alexnet  Evaluated 32/1000: Top 1 Accuracy: 53.125 Top 5 Accuracy: 75.000


100%|██████████| 32/32 [01:11<00:00,  2.22s/it]


...

  0%|          | 0.00/230M [00:00<?, ?B/s]

  3%|▎         | 1/32 [00:01<01:00,  1.96s/it]

resnet152  Evaluated 32/1000: Top 1 Accuracy: 81.250 Top 5 Accuracy: 93.750


100%|██████████| 32/32 [00:21<00:00,  1.48it/s]


resnet152  Evaluated 1000/1000: Top 1 Accuracy: 79.300 Top 5 Accuracy: 93.900
Done for resnet152_Original
